## Get `Intent`

In [1]:
import pickle
import numpy as np
import json
import random
import spacy
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [2]:
from keras.models import load_model

model = load_model('Model.h5')

intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

with open('classes.pkl','rb') as file:  
    classes = pickle.load(file)

with open('words.pkl','rb') as file:  
    words = pickle.load(file)

In [3]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words):
    
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                
    return(np.array(bag))

def predict_class(sentence, model):
    
    # filter out predictions below a threshold
    p = bow(sentence, words)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = {"intent":classes[r[0]] for r in results}
    return return_list

In [4]:
predict_class('What is the weather in cairo?',model)

{'intent': 'Weather'}

In [5]:
predict_class('Ask wikipedia about Electricity',model)

{'intent': 'Info_Request'}

-----------------------------------------------------------

## Get `Entities`

In [6]:
path = r"C:\Users\Mohamed Nabil\Documents\Project"
nlp = spacy.load(path)

def NER(sentence):
    doc = nlp(sentence)
    result = {ent.label_: ent.text for ent in doc.ents}        
    return result
    

In [7]:
NER('What is the weather in berlin?')

{'location': 'berlin'}

In [8]:
NER('play music for Cairokee') 

{'artist': 'Cairokee'}

In [9]:
NER('dim lights in bedroom')

{'light_off': 'dim', 'room_type': 'bedroom'}

In [10]:
NER('play insan by hamza namira')

{'song_name': 'insan', 'artist': 'hamza namira'}

In [11]:
NER('what is the weather')

{}

------------------------------


## Final Function That Return `NLU` For Utterance

In [12]:
def NLU(sentence):
    intent = predict_class(sentence,model)
    entity = NER(sentence)
    
    if entity == {}:
        result = {'Intent': intent['intent'],'Entities':None}
    else:
        result = {'Intent': intent['intent'],'Entities':entity}
        
    return result
    

In [13]:
NLU('set an alarm for seven am')

{'Intent': 'Alarm', 'Entities': {'time': 'seven am'}}

In [14]:
NLU('tell me how is the weather today')

{'Intent': 'Weather', 'Entities': {'date': 'today'}}

In [15]:
NLU('dim lights in bathroom')

{'Intent': 'Iot', 'Entities': {'light_off': 'dim', 'room_type': 'bathroom'}}

In [16]:
NLU('hoe is the weather tomorrow?')

{'Intent': 'Weather', 'Entities': {'date': 'tomorrow'}}

In [17]:
NLU('what is the weather')

{'Intent': 'Weather', 'Entities': None}

In [18]:
NLU('turn the lights off in bedroom')

{'Intent': 'Iot',
 'Entities': {'light_off': 'lights off', 'room_type': 'bedroom'}}

In [19]:
NLU('play shallow by lady gaga')

{'Intent': 'PlayMusic',
 'Entities': {'song_name': 'shallow', 'artist': 'lady gaga'}}